# Setup

In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.6 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=b346f3df03a4087509c6a5cfd4cb30242c1adf76e6b0190f08dece1e1e5f4252
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [2]:
import torch

device = torch.device("cpu")

In [3]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 480kB/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.27MB/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [6]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [7]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference
import random
random.seed(10)

# Train using JTT

In [8]:
#jtt params
jtt_epochs = 2
print('Total trainset size: ' + str(len(trainset)))
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)

Total trainset size: 48004


## Group Inference Merge: 2 Groups

In [9]:
def gi_subset(num_partitions, partition_id, trainset, device, shuffled_indices):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = shuffled_indices[partition_len*partition_id:partition_len*partition_id+partition_len]
  train_index = train_index.tolist()
  print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=True
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  jtt = JTTInference([pred_labels[i] for i in train_index], [actual_labels[i] for i in train_index])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  print('\nNominal Upsample: ' + str(nominal_upsample))
  print('Majority class size: ' + str(len(gi[(0, 0)])))
  print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [train_index[i] for i in gi[(0, 0)]]
  minority = [train_index[i] for i in gi[(0, 1)]]
  return majority, minority

In [10]:
maj1, min1 = gi_subset(4, 0, trainset, device, shuffled_indices)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[36725, 43002, 34866, 40548, 1729, 24993, 11200, 10557, 20402, 9957, 6444, 31320, 44170, 2533, 37078, 18296, 6907, 721, 32756, 8268, 38465, 32832, 32752, 8330, 10113, 31792, 47116, 9875, 46405, 10816, 18115, 44148, 23618, 2453, 17523, 20259, 36930, 6552, 30035, 2514, 27848, 22058, 10687, 32656, 12147, 10661, 27808, 27006, 14411, 43738, 36808, 679, 42887, 541, 15795, 42566, 47223, 24764, 8875, 2210, 18440, 13805, 12944, 26238, 44693, 39704, 5052, 1245, 46139, 32391, 31934, 157, 28410, 16110, 17863, 13254, 28427, 24141, 46960, 26389, 35728, 9772, 9768, 43015, 22579, 34075, 44230, 46397, 17269, 30268, 34441, 28796, 20117, 28306, 11028, 34728, 7773, 11057, 46098, 20742, 46576, 22026, 23483, 19482, 29054, 36889, 2045, 38201, 28674, 2531, 30729, 3310, 16546, 38284, 34978, 16371, 24583, 40124, 46088, 19769, 9305, 14887, 45888, 17497, 16008, 12980, 34872, 34878, 39707, 4672, 39289, 15273, 12786, 32914, 9029, 14740, 26231, 2483, 22396, 15492, 36697, 15354, 4213, 35324, 17487, 38908, 995, 20696,

Epoch 0:   4%|▎         | 7/188 [00:00<00:16, 10.83batch/s, accuracy=26.5625%, loss=1.62]

 | Epoch 0 | Loss: 1.6240710020065308 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6138420104980469 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.616407871246338 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.61793053150177 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6140621900558472 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.621171474456787 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.6208657026290894 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6024869680404663 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.6106466054916382 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6094075441360474 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6161991357803345 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6163698434829712 | Accuracy: 26.5625%


Epoch 0:  10%|█         | 19/188 [00:01<00:06, 27.90batch/s, accuracy=37.5%, loss=1.58]   

...

 | Epoch 1 | Loss: 0.18556146323680878 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.0069357892498373985 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006774567998945713 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007059842348098755 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006819772068411112 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007135563064366579 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0064134434796869755 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0073552499525249004 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.00757115613669157 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005575127433985472 | Accuracy: 100.0%


Epoch 1:  99%|█████████▉| 187/188 [00:03<00:00, 45.75batch/s, accuracy=100.0%, loss=0.00478]

 | Epoch 1 | Loss: 0.005640967283397913 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.11389327049255371 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.004779278300702572 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:08<00:00, 87.69batch/s] 


Nominal Upsample: 199.01666666666668
Majority class size: 11941
Minority class size: 60


In [11]:
maj2, min2 = gi_subset(4, 1, trainset, device, shuffled_indices)

[618, 29719, 17258, 31230, 30019, 34182, 40435, 37907, 25308, 27007, 2699, 40101, 46625, 23044, 22756, 28696, 38358, 45708, 38001, 22028, 46161, 40388, 5912, 12446, 23521, 16236, 2011, 5448, 42785, 6796, 35897, 32553, 36137, 19972, 11001, 14710, 23589, 35675, 28810, 12338, 29874, 34570, 1906, 7267, 20548, 7396, 35894, 3194, 18152, 45566, 37664, 9919, 11704, 42781, 16515, 42403, 635, 8111, 15510, 17991, 27721, 1530, 9648, 33887, 46664, 40279, 14127, 35244, 35164, 40502, 14068, 19658, 4974, 16674, 16847, 31663, 32048, 23274, 2972, 7195, 32949, 37778, 23905, 47997, 31715, 20895, 35754, 22314, 47368, 28144, 28042, 29253, 29227, 3234, 14692, 47317, 34954, 39524, 5621, 43017, 29746, 42883, 10659, 7797, 28697, 12305, 27210, 17470, 43357, 6096, 42442, 42790, 44867, 28309, 41794, 11276, 13908, 34841, 31790, 14083, 26147, 44553, 29841, 8537, 7662, 40975, 42877, 44856, 11430, 15395, 43443, 39995, 19187, 46644, 41297, 23233, 7479, 12194, 16954, 40524, 42411, 22910, 17547, 11381, 32646, 22327, 3601

Epoch 0:   1%|          | 1/188 [00:00<00:35,  5.31batch/s, accuracy=17.1875%, loss=1.63]

 | Epoch 0 | Loss: 1.6317832469940186 | Accuracy: 17.1875%


Epoch 0:   4%|▎         | 7/188 [00:00<00:06, 27.58batch/s, accuracy=20.3125%, loss=1.6]

 | Epoch 0 | Loss: 1.6156895160675049 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6170600652694702 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.619084358215332 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.606371521949768 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.613425612449646 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.621077060699463 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6068320274353027 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.599511742591858 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.603519082069397 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6010860204696655 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.5987842082977295 | Accuracy: 20.3125%


...

 | Epoch 1 | Loss: 0.10203943401575089 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.008028569631278515 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.23670245707035065 | Accuracy: 96.875%
 | Epoch 1 | Loss: 0.008974443189799786 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.13985022902488708 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.010128894820809364 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010426740162074566 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.01058149803429842 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.011078642681241035 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.32058003544807434 | Accuracy: 95.3125%


Epoch 1: 100%|██████████| 188/188 [00:03<00:00, 47.53batch/s, accuracy=100.0%, loss=0.0155]


 | Epoch 1 | Loss: 0.015516783110797405 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:08<00:00, 93.06batch/s] 


Nominal Upsample: 156.9078947368421
Majority class size: 11925
Minority class size: 76


In [12]:
maj3, min3 = gi_subset(4, 2, trainset, device, shuffled_indices)

[40292, 43686, 15013, 37894, 27982, 45005, 41279, 37626, 46679, 21826, 21012, 22804, 41057, 37255, 2031, 5176, 29424, 25663, 5101, 32990, 42418, 14428, 2875, 31133, 18597, 39740, 38168, 5077, 47289, 4171, 18496, 21476, 16946, 2163, 3938, 35595, 23427, 20042, 42505, 30667, 5531, 9385, 1014, 33226, 4447, 20868, 9546, 16762, 8887, 8922, 27807, 31758, 41520, 27336, 16906, 28983, 42357, 27371, 39495, 3906, 3691, 16202, 20650, 12741, 21509, 17965, 10862, 8376, 23836, 35297, 36124, 18643, 10214, 30693, 28788, 24544, 28676, 41513, 21641, 34952, 27733, 16598, 298, 33868, 7642, 10297, 21741, 33936, 14655, 21658, 23093, 33966, 5606, 6171, 35828, 47494, 43622, 45870, 37914, 36454, 31976, 29486, 19900, 18604, 45982, 33360, 28932, 2482, 35933, 14343, 19665, 24133, 37539, 43964, 36882, 44258, 37667, 38571, 19534, 43194, 26008, 18898, 37157, 14507, 25775, 29819, 1993, 41020, 26395, 47934, 16614, 31402, 36408, 43864, 33872, 31073, 20212, 44100, 7954, 26371, 47788, 43455, 31923, 11335, 15884, 50, 20641,

Epoch 0:   4%|▍         | 8/188 [00:00<00:05, 30.51batch/s, accuracy=23.4375%, loss=1.61]

 | Epoch 0 | Loss: 1.6243271827697754 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.6148546934127808 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6195591688156128 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6191473007202148 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.603568196296692 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6057027578353882 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.6158305406570435 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6210647821426392 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6080948114395142 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6078063249588013 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6039639711380005 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.6009222269058228 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.61117684841156 | Accuracy: 23.4375%


Epoch 0:  11%|█         | 21/188 [00:00<00:03, 49.55batch/s, accuracy=56.25%, loss=1.58]  

 | Epoch 0 | Loss: 1.6087244749069214 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.5999456644058228 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.5963894128799438 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.58971107006073 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.5996320247650146 | Accuracy: 45.3125%
 | Epoch 0 | Loss: 1.594499111175537 | Accuracy: 57.8125%
 | Epoch 0 | Loss: 1.5871391296386719 | Accuracy: 64.0625%
 | Epoch 0 | Loss: 1.602320671081543 | Accuracy: 40.625%
 | Epoch 0 | Loss: 1.5976150035858154 | Accuracy: 46.875%
 | Epoch 0 | Loss: 1.5877326726913452 | Accuracy: 56.25%
 | Epoch 0 | Loss: 1.5930275917053223 | Accuracy: 43.75%
 | Epoch 0 | Loss: 1.5812175273895264 | Accuracy: 57.8125%
 | Epoch 0 | Loss: 1.5821306705474854 | Accuracy: 56.25%


...

 | Epoch 1 | Loss: 0.009359222836792469 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.008638935163617134 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0086967246606946 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007689256686717272 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006970662623643875 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006877372972667217 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.006522300653159618 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005854169372469187 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.12175017595291138 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.00497416639700532 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.1210363507270813 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.004902837332338095 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004329772666096687 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.00477443914860487 | Accuracy: 100.0%


Epoch 1: 100%|██████████| 188/188 [00:03<00:00, 54.72batch/s, accuracy=100.0%, loss=0.00439]


 | Epoch 1 | Loss: 0.00441321823745966 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004048572853207588 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004393246956169605 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:08<00:00, 93.60batch/s] 


Nominal Upsample: 163.3972602739726
Majority class size: 11928
Minority class size: 73


In [13]:
maj4, min4 = gi_subset(4, 3, trainset, device, shuffled_indices)

[16385, 4783, 8228, 13308, 3259, 25678, 43370, 28324, 16179, 28876, 32895, 21383, 45178, 41694, 43225, 33934, 11780, 8690, 31738, 30719, 955, 11879, 12010, 31574, 29618, 26380, 20883, 29527, 32127, 21550, 44302, 43681, 6554, 27009, 43629, 34592, 25630, 17377, 12263, 24237, 44579, 34321, 31613, 29269, 26, 17726, 37001, 36788, 12548, 22775, 4668, 1722, 42929, 7763, 32944, 26206, 16910, 28017, 8199, 21816, 4023, 47178, 25606, 6039, 33113, 17999, 45881, 31399, 1822, 33314, 40311, 7013, 11265, 24934, 20563, 3377, 40667, 8912, 29311, 29554, 3239, 1070, 22052, 5264, 45821, 22389, 38298, 25341, 7206, 17046, 14506, 22908, 30687, 26188, 46404, 27658, 29387, 10990, 39765, 24207, 37850, 31568, 8726, 19577, 37117, 34152, 25160, 45586, 22251, 37783, 42903, 13744, 36497, 39014, 9652, 2012, 28604, 15280, 27483, 39101, 11612, 19080, 32536, 22486, 28537, 13411, 33224, 46953, 13239, 28065, 47141, 21287, 18825, 36207, 41058, 17251, 38529, 5878, 31678, 8496, 35729, 5133, 5359, 17918, 19940, 18908, 24736, 1

Epoch 0:   4%|▎         | 7/188 [00:00<00:06, 27.79batch/s, accuracy=17.1875%, loss=1.61]

 | Epoch 0 | Loss: 1.6143487691879272 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6207995414733887 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6208603382110596 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6299916505813599 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6234588623046875 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.6190853118896484 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.608705759048462 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6047216653823853 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.607518196105957 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6082059144973755 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6127442121505737 | Accuracy: 9.375%


Epoch 0:  10%|█         | 19/188 [00:00<00:03, 46.99batch/s, accuracy=9.375%, loss=1.59]  

 | Epoch 0 | Loss: 1.6058775186538696 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6002448797225952 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6001555919647217 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.6028692722320557 | Accuracy: 43.75%
 | Epoch 0 | Loss: 1.596914291381836 | Accuracy: 45.3125%
 | Epoch 0 | Loss: 1.5955270528793335 | Accuracy: 43.75%
 | Epoch 0 | Loss: 1.5947668552398682 | Accuracy: 42.1875%
 | Epoch 0 | Loss: 1.5961906909942627 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.592968225479126 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.5916647911071777 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.589157223701477 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.5943070650100708 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.585090160369873 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.5914247035980225 | Accuracy: 9.375%


...

 | Epoch 1 | Loss: 0.012233237735927105 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010217508301138878 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010687196627259254 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.008468354120850563 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.00950573943555355 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.24092186987400055 | Accuracy: 96.875%
 | Epoch 1 | Loss: 0.11206204444169998 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.09316740185022354 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.009558027610182762 | Accuracy: 100.0%


Epoch 1:  99%|█████████▉| 187/188 [00:03<00:00, 51.05batch/s, accuracy=100.0%, loss=0.00605]

 | Epoch 1 | Loss: 0.009355999529361725 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.008355682715773582 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010458864271640778 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.009369472973048687 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007994567975401878 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.009090999141335487 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007578209973871708 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.10975795984268188 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.007151832804083824 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.2663091719150543 | Accuracy: 96.875%
 | Epoch 1 | Loss: 0.006052428390830755 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:07<00:00, 101.47batch/s]


Nominal Upsample: 145.35365853658536
Majority class size: 11919
Minority class size: 82


In [14]:
min = min1.copy()
min.extend(min2)
min.extend(min3)
min.extend(min4)
maj = maj1.copy()
maj.extend(maj2)
maj.extend(maj3)
maj.extend(maj4)
upsampled_indices = [i for i in min for x in range(100)]
upsampled_indices.extend(maj)
len(upsampled_indices)

76813

## Robust Training

In [15]:
from spuco.robust_train import CustomSampleERM

robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
rt_erm = CustomSampleERM(
    model=robust_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=3,
    indices=upsampled_indices,
    device=device,
    verbose=True
)
rt_erm.train()

Epoch 0:   1%|          | 8/1201 [00:00<00:44, 26.63batch/s, accuracy=15.625%, loss=1.6]

 | Epoch 0 | Loss: 1.6084458827972412 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6207349300384521 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6231954097747803 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6129820346832275 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6022049188613892 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6199524402618408 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.619279146194458 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6309422254562378 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6075048446655273 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6125075817108154 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6259030103683472 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6024447679519653 | Accuracy: 15.625%


Epoch 0:   2%|▏         | 20/1201 [00:00<00:28, 41.74batch/s, accuracy=34.375%, loss=1.6]

 | Epoch 0 | Loss: 1.6041165590286255 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6040292978286743 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6109495162963867 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6102900505065918 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6121217012405396 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6151989698410034 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.610500693321228 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.599136471748352 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6014056205749512 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6041969060897827 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6027231216430664 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6014056205749512 | Accuracy: 34.375%


Epoch 0:   3%|▎         | 34/1201 [00:00<00:21, 54.71batch/s, accuracy=31.25%, loss=1.6]  

...

Epoch 2:  99%|█████████▉| 1192/1201 [00:26<00:00, 48.79batch/s, accuracy=100.0%, loss=0.000341]

 | Epoch 2 | Loss: 0.003810187103226781 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0011755048763006926 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0019038927275687456 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.005899775307625532 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.00044416473247110844 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0005367450066842139 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0009554778807796538 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0004931454895995557 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0009275740012526512 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0001582781260367483 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0003409273049328476 | Accuracy: 100.0%


Epoch 2: 100%|█████████▉| 1198/1201 [00:26<00:00, 49.89batch/s, accuracy=100.0%, loss=0.0057]  

 | Epoch 2 | Loss: 0.003926011733710766 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0007833968847990036 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0006281975656747818 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0031240698881447315 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0002901707775890827 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.010431632399559021 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.00044891596189700067 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.005699263885617256 | Accuracy: 100.0%


Epoch 2: 100%|██████████| 1201/1201 [00:26<00:00, 44.64batch/s, accuracy=100.0%, loss=0.0057]


In [16]:
from spuco.evaluate import Evaluator

rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=True
)
rt_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:06,  3.50it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:06,  3.36it/s]

Group (0, 1) Accuracy: 69.97635933806147


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.49it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:07<00:00,  3.04it/s]

Group (4, 3) Accuracy: 66.41414141414141


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  3.08it/s]

Group (4, 4) Accuracy: 99.4949494949495


{(0, 0): 100.0,
 (0, 1): 69.97635933806147,
 (0, 2): 84.86997635933807,
 (0, 3): 82.97872340425532,
 (0, 4): 72.10401891252955,
 (1, 0): 68.45965770171149,
 (1, 1): 99.51100244498778,
 (1, 2): 76.7156862745098,
 (1, 3): 55.63725490196079,
 (1, 4): 55.3921568627451,
 (2, 0): 69.6,
 (2, 1): 46.666666666666664,
 (2, 2): 99.73333333333333,
 (2, 3): 49.06666666666667,
 (2, 4): 26.203208556149733,
 (3, 0): 63.5678391959799,
 (3, 1): 58.69017632241814,
 (3, 2): 81.10831234256926,
 (3, 3): 99.74811083123426,
 (3, 4): 69.52141057934509,
 (4, 0): 79.34508816120906,
 (4, 1): 31.486146095717885,
 (4, 2): 32.49370277078086,
 (4, 3): 66.41414141414141,
 (4, 4): 99.4949494949495}

In [17]:
rt_evaluator.worst_group_accuracy

((2, 4), 26.203208556149733)

In [18]:
rt_evaluator.average_accuracy

99.48051518608361

In [19]:
rt_evaluator.evaluate_spurious_attribute_prediction()

20.02